In [1]:
import textacy

ModuleNotFoundError: No module named 'textacy'

In [4]:
import re
import textacy
from nltk.corpus import wordnet
from fuzzywuzzy import fuzz

def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name())
    return synonyms

def fuzzy_match(keyword, text, threshold=80):
    words = text.split()
    matched_words = [word for word in words if fuzz.ratio(keyword, word) >= threshold]
    return matched_words

def modified_kwic(doc, keyword, window_width=5, ignore_case=True, threshold=80):
    # Get plurals, synonyms, and fuzzy matches
    plurals = {f"{keyword}s", f"{keyword}es"}
    synonyms = get_synonyms(keyword)
    fuzzy_matches = fuzzy_match(keyword, doc.text, threshold)

    # Create a regular expression pattern for the search term
    search_terms = set([keyword]).union(plurals).union(synonyms).union(fuzzy_matches)
    search_terms_pattern = r"|".join(re.escape(term) for term in search_terms)

    # Use the modified pattern in the kwic function
    return textacy.extract.kwic(doc, search_terms_pattern, window_width, ignore_case)

In [20]:

import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from yellowbrick.text import TSNEVisualizer

plt.style.use('fivethirtyeight')
sns.set_context("notebook", rc={"font.size":16,
                                "axes.titlesize":20,
                                "axes.labelsize":18})


import re
import string

from collections import Counter
from spellchecker import SpellChecker
import contractions
from num2words import num2words
import emoji
import random
from textacy import extract


import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from yellowbrick.text.freqdist import FreqDistVisualizer

from gensim.models import Word2Vec


from tqdm import tqdm
tqdm.pandas()


from termcolor import colored


import warnings
warnings.filterwarnings("ignore")


from typing import List, Tuple


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MokS\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\MokS\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\MokS\AppData\Roaming\nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\MokS\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


In [41]:
from pypdf import PdfReader
reader = PdfReader("paper.pdf")
text = []

def visitor_body(text, cm, tm, font_dict, font_size):
    y = tm[5]
    if y > 50 and y < 720:
        parts.append(text)

for page in reader.pages:
    parts = []
    page.extract_text(visitor_text=visitor_body)
    text.append("".join(parts))
 
    
df_train=pd.Series(text)  
df_train


0     Citation: Bruijns, B.; Knotter, J.;\nTiggelaar...
1     within conventional STR proﬁling since the bes...
2     truly became commercially available. A systema...
3     review are 8, 21, 12, and 2 for ‘ParaDNA,’ ‘Ra...
4     samples, (h) perform well (sensitivity and sel...
5     used as a presumptive test for samples with me...
6     chemistry is available as ACE NGM SElect Expre...
7     Samples such as cable ties, fabric, matchstick...
8     Shackleton et al. used the AmpFlSTR1 NGMSElect...
9     data generated showed that the workﬂow of Rapi...
10    Figure 5. Schematic overview of the ANDE I-chi...
11    ANDE 6C with the I-Chip to address the FBI’s Q...
12    Figure 6. Overview of the MiDAS cartridge for ...
13    \nTable 1. Characteristics of the ParaDNA, Rap...
14    system is more robust in terms of resistance t...
15    C until use) near the testing area, while the ...
16    \n Time-to-result : although it is not (yet) ...
17    , 26–37. [CrossRef]\n4. Wiley, R.; Sage, K

In [31]:
def lower_case(text: str) -> str:
    return text.lower()

def remove_spaces_tabs(text: str) -> str:
    return " ".join(text.split())

def remove_punct(text: str) -> str:
    translator = str.maketrans("", "", string.punctuation)
    return text.translate(translator)

def remove_single_char(text: str) -> str:
    return re.sub(r"\b[a-zA-Z]\b", "", text)

def remove_html(text: str) -> str:
    html = re.compile(r"<.*?>")
    return html.sub(r"", text)

def remove_url(text: str) -> str:
    url = re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"", text)

def remove_emoji(text: str) -> str:
    # Reference : https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F1E0-\U0001F1FF"  # flags (iOS)
        "]+",
        flags=re.UNICODE,
    )
    return emoji_pattern.sub(r"", text)

def remove_stopwords(text: str) -> str:
    stop_words = set(stopwords.words("english"))
    stop_words.update(["et", "al", "et al"]) # add custom stopwords
    stop_words -= {"no", "not"} # remove custom stopwords
    word_tokens = word_tokenize(text)
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    return " ".join(filtered_sentence)

def expand_contractions(text: str) -> str:
    return contractions.fix(text)

def lemmatize_text_custom(text: str, lemmatizer) -> str:
    wordnet_map = {
        "J": wordnet.ADJ,
        "N": wordnet.NOUN,
        "V": wordnet.VERB,
        "R": wordnet.ADV,
    }

    w_pos_tags = nltk.pos_tag(text.split())
    lemmatized_output = " ".join([lemmatizer.lemmatize(w, wordnet_map.get(pos[0], wordnet.NOUN)) for w, pos in w_pos_tags])
    return lemmatized_output

def stem_text_custom(text: str, stemmer) -> str:
    word_tokens = word_tokenize(text)
    stemmed_output = " ".join([stemmer.stem(w) for w in word_tokens])
    return stemmed_output
def correct_spelling(text: str) -> str:
    spell = SpellChecker()
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        if word in misspelled_words:
            # if correction is none return the original word
            if spell.correction(word) is not None:
                corrected_text.append(spell.correction(word))
            else:
                corrected_text.append(word)
        else:
            corrected_text.append(word)
    return " ".join(corrected_text)




def prepare(text, pipeline, lemmatizer=None, stemmer=None):
    tokens = text
    for transform in pipeline:
        # if lemmatize or stem function pass in, perform transformation
        if transform.__name__ == "lemmatize_text_custom":
            tokens = transform(tokens, lemmatizer)
        elif transform.__name__ == "stem_text_custom":
            tokens = transform(tokens, stemmer)
        else:
            tokens = transform(tokens)

    return tokens



In [42]:
pipeline = [
    lower_case,
    expand_contractions,
    remove_spaces_tabs,
    remove_url,
    remove_punct,
    remove_single_char,
    remove_html,
    remove_stopwords,
    expand_contractions,
    correct_spelling,
    
]

df_train_clean = df_train.progress_apply(prepare, pipeline=pipeline)
df_train_clean

100%|██████████| 21/21 [10:12<00:00, 29.19s/it]


0     citation bruins knotted tiggelaar systematic r...
1     within conventional star prowling since best r...
2     truly became commercially available systematic...
3     review 8 21 12 2 i parade i i rapidity i i and...
4     samples perform well sensitivity selectively n...
5     used presumptive test samples medium high temp...
6     chemistry available ace nom select express sam...
7     samples cable ties fabric matchstick ziplock b...
8     shackleton used ampflstr1 ngmselect express st...
9     data generated showed workflow rapidity genera...
10    figure 5 schematic overview and chip top view ...
11    and ac chip address fbi i quality assurance st...
12    figure 6 overview midas cartridge dna extracti...
13    table 1 characteristics parade rapidity 200 ra...
14    system robust terms resistance shocks upon tra...
15    use near testing area chip and could stored ro...
16    i timetoresult although not yet possible gener...
17    26–37 crossed 4 wiley sage large budowle i

In [44]:
df_train_clean

0     citation bruins knotted tiggelaar systematic r...
1     within conventional star prowling since best r...
2     truly became commercially available systematic...
3     review 8 21 12 2 i parade i i rapidity i i and...
4     samples perform well sensitivity selectively n...
5     used presumptive test samples medium high temp...
6     chemistry available ace nom select express sam...
7     samples cable ties fabric matchstick ziplock b...
8     shackleton used ampflstr1 ngmselect express st...
9     data generated showed workflow rapidity genera...
10    figure 5 schematic overview and chip top view ...
11    and ac chip address fbi i quality assurance st...
12    figure 6 overview midas cartridge dna extracti...
13    table 1 characteristics parade rapidity 200 ra...
14    system robust terms resistance shocks upon tra...
15    use near testing area chip and could stored ro...
16    i timetoresult although not yet possible gener...
17    26–37 crossed 4 wiley sage large budowle i

In [45]:
def get_top_ngrams(text: pd.Series, ngram: int =1, top_n: int =10) -> List[Tuple[str, int]]:
    vec = CountVectorizer(ngram_range=(ngram, ngram), stop_words="english").fit(text)
    bag_of_words = vec.transform(text)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)
    return words_freq[:top_n]

get_top_ngrams(df_train_clean, ngram=2)

[('rapid dna', 76),
 ('rapidity id', 57),
 ('dna analysis', 52),
 ('rapidity 200', 48),
 ('forensic sci', 42),
 ('globalfiler express', 39),
 ('parade rapidity', 35),
 ('nom select', 35),
 ('ampflstr nom', 33),
 ('select express', 33)]

In [ ]:


import re
import textacy
from nltk.corpus import wordnet
from fuzzywuzzy import fuzz

def search_keyword(text: str, keyword: str) -> List:
    return list(extract.keyword_in_context(text, keyword, window_width=50, ignore_case=True))

def search_keyword_in_df(text: pd.Series, keyword: str, n: int = 10) -> str:

    # apply search keyword function to each row
    kic = text.apply(search_keyword, keyword=keyword)
    
    # filter out empty lists
    kic = kic[kic.apply(len) > 0]

    # check if there are any results
    if len(kic) > 0:
        # check if n is greater than the number of samples
        if len(kic) < n:
            n = len(kic)
        sample_list = kic.sample(n)

        # print results
        for sample in sample_list:
            print(f"{sample[0][0]} {colored(sample[0][1], 'red')} {sample[0][2]}")
    else:
        print(f"No samples found for {colored(keyword, 'red')}")

def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name())
    return synonyms

def fuzzy_match(keyword, text, threshold=80):
    words = text.split()
    matched_words = [word for word in words if fuzz.ratio(keyword, word) >= threshold]
    return matched_words

def modified_kwic(doc, keyword, window_width=5, ignore_case=True, threshold=80):
    # Get plurals, synonyms, and fuzzy matches
    plurals = {f"{keyword}s", f"{keyword}es"}
    synonyms = get_synonyms(keyword)
    fuzzy_matches = fuzzy_match(keyword, doc.text, threshold)

    # Create a regular expression pattern for the search term
    search_terms = set([keyword]).union(plurals).union(synonyms).union(fuzzy_matches)
    search_terms_pattern = r"|".join(re.escape(term) for term in search_terms)

    # Use the modified pattern in the kwic function
    return textacy.extract.kwic(doc, search_terms_pattern, window_width, ignore_case)